In [17]:
import pandas as pd
import numpy as np
from sklearn.metrics import make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder

In [2]:
def mape(y, y_pred):
    return np.mean(np.absolute(np.divide(np.subtract(y, y_pred), y)))

In [46]:
players = pd.read_csv("../data/player_data.csv")
train = pd.read_csv("../data/training_set.csv").drop("Game_ID", axis = 1)
train_y = train.pop("Rounded Viewers")
test = pd.read_csv("../data/test_set.csv").drop("Total_Viewers", axis = 1)
ids = test.pop("Game_ID")
game_data = pd.read_csv("../data/game_data.csv")

In [52]:
t1 = pd.get_dummies(train, columns = ["Away_Team", "Home_Team", "Country"])
t1["Season"] = le.fit_transform(t1["Season"])
t1["Game_Date"] = le.fit_transform(t1["Game_Date"])
test["Season"] = le.transform(test["Season"])
test["Game_Date"] = le.transform(test["Game_Date"])
model = RandomForestRegressor(n_estimators = 100)
#model.fit(train, train_y)
test

,Season,Game_Date,Away_Team,Home_Team
0,2016-17,10/26/2016,DEN,NOP
1,2016-17,10/28/2016,HOU,DAL
2,2016-17,10/29/2016,BOS,CHA
3,2016-17,10/29/2016,ATL,PHI
4,2016-17,11/1/2016,LAL,IND
5,2016-17,11/1/2016,UTA,SAS
6,2016-17,11/1/2016,NYK,DET
7,2016-17,11/1/2016,SAC,MIA
8,2016-17,11/2/2016,CHI,BOS
9,2016-17,11/2/2016,POR,PHX
